In [ ]:
from preprocess import ProductClassificationPipeline, PipelineConfig
from utils import generate_prediction_report, analyze_prediction_errors, plot_prediction_distribution
from data.processed_data.executer_pour_telecharger_donnees import telecharger_et_extraire_zip
import pandas as pd
import os
import sys
import yaml

url = "https://drive.google.com/file/d/1guhuHp0dVRPWCtZ7570jEsTub6m2RrRF/view?usp=sharing"
fichier_zip = "Preprocessed_data.zip"
dossier_donnees_pretraitees = "data/processed_data"
fichier_donnees_pretraitees = "data/processed_data/y_train.npz"


def load_model_configs():
    """
    Charge les configurations des modèles depuis le YAML
    
    Returns:
        dict: Configurations des modèles avec leurs paramètres
    """
    try:
        config_path = os.path.join('data', 'models', 'model_configs.yaml')
        with open(config_path, 'r', encoding='utf-8') as f:
            configs = yaml.safe_load(f)
        
        # Vérification de la présence de tous les modèles attendus
        expected_models = {'xgboost', 'neural_net'}
        missing_models = expected_models - set(configs.keys())
        
        if missing_models:
            print(f"Attention : modèles manquants dans la configuration : {missing_models}")
            
        return configs
        
    except Exception as e:
        print(f"Erreur lors du chargement des configurations : {str(e)}")
        return {}

if __name__ == "__main__":
    # Forcer le prétraitement des données ou les télécharger si souhaité
    force_preprocess = False # Mettre à True pour forcer le preprocesing des données
    
    if not force_preprocess and not os.path.exists(fichier_donnees_pretraitees):        
        # Appel de la fonction pour télécharger + extraire
        telecharger_et_extraire_zip(
            url=url,
            fichier_zip=fichier_zip,
            dossier_extraction=dossier_donnees_pretraitees
        )
        
    # Chargement des configurations
    config = PipelineConfig.from_yaml('config.yaml')
    pipeline = ProductClassificationPipeline(config)
    
    #Prétraitement
    try:
        pipeline.prepare_data(force_preprocess=force_preprocess)
    except Exception as e:
        print(f"Erreur lors du prétraitement : {str(e)}")
        sys.exit(1)
        
    # Chargement des configurations des modèles depuis le YAML
    models_to_test = load_model_configs()
    
    # Initialisation des dictionnaires pour stocker tous les résultats
    all_results = {}
    
    for model_name, params in models_to_test.items():
        print(f"\nTraitement  de {model_name}")
        recherche_hyperparametres = False
        if recherche_hyperparametres:
            print(f"\nRecherche des hyperparamètres pour {model_name}")            
            # Optimisation des hyperparamètres
            if model_name == 'xgboost':
                best_params = pipeline.optimize_hyperparameters('xgboost')
                pipeline.save_hyperparameters('xgboost', best_params)
                params.update(best_params)
            elif model_name == 'neural_net':
                best_params = pipeline.optimize_hyperparameters('neural_net')
                pipeline.save_hyperparameters('neural_net', best_params)
                params.update(best_params)
    
        # Vérification des prédictions existantes
        if pipeline.predictions_exist(model_name):
            predictions, probabilities = pipeline.load_predictions(model_name)
            pipeline.load_model(model_name)
            print(f"Prédictions chargées pour {model_name}")
        else:
            # Chargement ou entraînement du modèle
            model_dir = os.path.join(pipeline.config.model_path, model_name)
            if os.path.exists(model_dir):
                try:
                    pipeline.load_model(model_name)
                    print(f"Modèle {model_name} chargé")
                except Exception as e:
                    print(f"Erreur chargement {model_name}: {e}")
                    continue
            else:
                pipeline.train_model(model_type=model_name, **params)

            # Prévisions
            predictions, probabilities = pipeline.predict(pipeline.preprocessed_data['X_test_split'])
                        
            # Création du DataFrame de sortie
            pipeline.save_predictions(model_name, predictions, probabilities)

        file_path_rapport = os.path.join('data', 'rapports', f'rapport_{model_name}.csv')
        file_path_erreurs = os.path.join('data', 'erreurs', f'erreurs_{model_name}.csv')
        if os.path.exists(file_path_rapport):
            print(f"Le rapport {model_name} existe.")
        if os.path.exists(file_path_erreurs):
            print(f"Le fichier erreurs {model_name} existe.")
            
        evalute = True #Forcer l'évaluation (Maj des résultats)
        plot = False   #Pour ne pas afficher le graphe de distribution des prédictions déjà généré
        if not os.path.exists(file_path_erreurs) or not os.path.exists(file_path_rapport) or evalute==True:    
            # Prévisions et Évaluation
            all_results[model_name] = pipeline.evaluate()
            plot= True
            print(f"\nRésultats pour le modèle {model_name}:\n {all_results[model_name]}")

        # Création du DataFrame de résultats
        results_df = pd.DataFrame.from_dict(all_results, orient='index')
        
        # Création du dossier results s'il n'existe pas
        results_dir = os.path.join('data', 'results')
        os.makedirs(results_dir, exist_ok=True)
        
        # Sauvegarde des résultats en CSV
        results_path = os.path.join(results_dir, 'models_comparaison_results.csv')
        results_df.to_csv(results_path)
        
        # Génération des rapports pour chaque modèle
        if not os.path.exists(file_path_rapport):
            rapport = generate_prediction_report(pipeline, predictions, probabilities)
            rapport.to_csv(file_path_rapport, index=False)
            # Visualisation
            plot_prediction_distribution(rapport, model_name)
        if not os.path.exists(file_path_erreurs):
            erreurs = analyze_prediction_errors(
                pipeline,
                predictions,
                pipeline.preprocessed_data['y_test_split']
            )
            erreurs.to_csv(file_path_erreurs, index=False)
            
    if evalute:
        # Affichage des résultats
        print("\nRésultats de la comparaison:")
        print("-" * 50)
        for model_name, metrics in all_results.items():
            print(f"\n{model_name}:")
            for metric_name, value in metrics.items():
                if isinstance(value, (int, float)):
                    print(f"{metric_name}: {value:.4f}")

/home/elion/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU disponible : Quadro M1200
CUDA Memory: 0.00GB / 0.00GB


2025-05-15 10:56:38,587 - classification_pipeline - INFO - Chargement des features pré-calculées...
2025-05-15 10:56:39,288 - classification_pipeline - INFO - Chargement effectué avec succès.



Traitement  de xgboost


2025-05-15 10:56:39,630 - classification_pipeline - INFO - Modèle xgboost chargé depuis ./data/models/xgboost
2025-05-15 10:56:39,684 - classification_pipeline - INFO - Prédiction avec modèle ML


Modèle xgboost chargé
Type de X: <class 'numpy.ndarray'>
Shape de X: (16984, 2048)
Type des données: float32
Shape après conversion: (16984, 2048)


/home/elion/.pyenv/versions/3.10.12/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:56:39] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
2025-05-15 10:56:53,443 - classification_pipeline - INFO - Prédiction avec modèle ML


Le rapport xgboost existe.
Le fichier erreurs xgboost existe.
X_test shape: ()
Type de X: <class 'numpy.ndarray'>
Shape de X: (16984, 2048)
Type des données: float32
Shape après conversion: (16984, 2048)
y_test shape: (16984,)

Résultats pour le modèle xgboost:
 {'accuracy': 0.619, 'macro_precision': 0.595, 'macro_recall': 0.602, 'macro_f1': 0.59, 'weighted_precision': 0.638, 'weighted_recall': 0.619, 'weighted_f1': 0.622, 'precision': array([0.518, 0.519, 0.406, 0.629, 0.557, 0.891, 0.605, 0.478, 0.322,
       0.665, 0.724, 0.336, 0.471, 0.689, 0.814, 0.507, 0.519, 0.592,
       0.75 , 0.696, 0.556, 0.711, 0.421, 0.879, 0.357, 0.718, 0.737]), 'recall': array([0.618, 0.582, 0.467, 0.675, 0.691, 0.865, 0.32 , 0.293, 0.355,
       0.723, 0.702, 0.508, 0.532, 0.505, 0.807, 0.689, 0.541, 0.352,
       0.689, 0.613, 0.542, 0.676, 0.506, 0.711, 0.555, 0.848, 0.885]), 'f1': array([0.564, 0.548, 0.434, 0.651, 0.617, 0.877, 0.419, 0.363, 0.338,
       0.693, 0.713, 0.404, 0.5  , 0.583, 0.81 , 0

2025-05-15 10:57:08,253 - classification_pipeline - INFO - Modèle neural_net chargé depuis ./data/models/neural_net


Modèle neural_net chargé
Type de X: <class 'numpy.ndarray'>
Shape de X: (16984, 2048)
Type des données: float32
Shape après conversion: (16984, 2048)
Le rapport neural_net existe.
Le fichier erreurs neural_net existe.
X_test shape: ()
Type de X: <class 'numpy.ndarray'>
Shape de X: (16984, 2048)
Type des données: float32
Shape après conversion: (16984, 2048)
y_test shape: (16984,)

Résultats pour le modèle neural_net:
 {'accuracy': 0.624, 'macro_precision': 0.586, 'macro_recall': 0.613, 'macro_f1': 0.595, 'weighted_precision': 0.641, 'weighted_recall': 0.624, 'weighted_f1': 0.627, 'precision': array([0.517, 0.539, 0.426, 0.64 , 0.623, 0.87 , 0.477, 0.479, 0.329,
       0.702, 0.635, 0.425, 0.503, 0.646, 0.783, 0.55 , 0.521, 0.477,
       0.716, 0.716, 0.529, 0.742, 0.357, 0.911, 0.358, 0.723, 0.641]), 'recall': array([0.599, 0.518, 0.536, 0.717, 0.68 , 0.901, 0.399, 0.343, 0.336,
       0.724, 0.758, 0.516, 0.551, 0.537, 0.844, 0.714, 0.517, 0.442,
       0.711, 0.562, 0.585, 0.66 , 0.5